# Offshore Graph Analysis - Exploratory Data Analysis

This notebook demonstrates connection to Neo4j and basic graph statistics for the Offshore Leaks dataset.

## 1. Setup and Installation

In [ ]:
# Install required packages if not already installed
# !pip install neo4j pandas matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from neo4j import GraphDatabase
import warnings
warnings.filterwarnings('ignore')

## 2. Connect to Neo4j Database

In [ ]:
# Connection parameters
URI = "bolt://localhost:7687"
USER = "neo4j"
PASSWORD = "password"

# Create driver
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# Verify connectivity
try:
    driver.verify_connectivity()
    print("✓ Successfully connected to Neo4j")
except Exception as e:
    print(f"✗ Connection failed: {e}")

## 3. Query 1: Basic Statistics - Count Nodes by Type

In [ ]:
# Query to count nodes by type
query = """
MATCH (n)
RETURN labels(n)[0] as node_type, count(n) as count
ORDER BY count DESC
"""

with driver.session() as session:
    result = session.run(query)
    data = [record for record in result]

# Create DataFrame
df_stats = pd.DataFrame([{"Node Type": r["node_type"], "Count": r["count"]} for r in data])

print("\n=== Node Statistics ===")
print(df_stats.to_string(index=False))

# Display as DataFrame
df_stats

## 4. Query 2: Top 10 Jurisdictions by Number of Entities

In [ ]:
# Query for top jurisdictions
query = """
MATCH (e:Entity)-[:REGISTERED_IN]->(j:Jurisdiction)
WITH j, count(e) as entity_count
ORDER BY entity_count DESC
LIMIT 10
RETURN j.name as jurisdiction, entity_count
"""

with driver.session() as session:
    result = session.run(query)
    data = [record for record in result]

# Create DataFrame
df_jurisdictions = pd.DataFrame([{
    "Jurisdiction": r["jurisdiction"],
    "Entity Count": r["entity_count"]
} for r in data])

print("\n=== Top 10 Jurisdictions by Entity Count ===")
print(df_jurisdictions.to_string(index=False))

df_jurisdictions

In [ ]:
# Create bar chart
plt.figure(figsize=(12, 6))
plt.barh(df_jurisdictions["Jurisdiction"], df_jurisdictions["Entity Count"], color='steelblue')
plt.xlabel('Number of Entities', fontsize=12)
plt.ylabel('Jurisdiction', fontsize=12)
plt.title('Top 10 Jurisdictions by Number of Entities', fontsize=14, fontweight='bold')
plt.gca().invert_yaxis()  # Show highest at top
plt.tight_layout()
plt.show()

## 5. Query 3: Risk Analysis - Top 5 Red Flag Addresses

In [ ]:
# Query for addresses with most registered entities (Red Flags)
query = """
MATCH (a:Address)<-[:REGISTERED_ADDRESS]-(e:Entity)
WITH a, count(e) as entity_count, collect(e.name)[0..5] as sample_entities
ORDER BY entity_count DESC
LIMIT 5
RETURN a.address as address, entity_count, sample_entities
"""

with driver.session() as session:
    result = session.run(query)
    data = [record for record in result]

# Create DataFrame
df_redflags = pd.DataFrame([{
    "Address": r["address"][:100] + "..." if r["address"] and len(r["address"]) > 100 else (r["address"] or "N/A"),
    "Entity Count": r["entity_count"],
    "Sample Entities": ", ".join([e for e in r["sample_entities"] if e])[:100]
} for r in data])

print("\n=== Top 5 Red Flag Addresses (Most Entities) ===")
print(df_redflags.to_string(index=False))

df_redflags

In [ ]:
# Visualize red flags
plt.figure(figsize=(14, 6))
plt.bar(range(len(df_redflags)), df_redflags["Entity Count"], color='crimson')
plt.xlabel('Address Index', fontsize=12)
plt.ylabel('Number of Entities', fontsize=12)
plt.title('Top 5 Red Flag Addresses - Entities per Address', fontsize=14, fontweight='bold')
plt.xticks(range(len(df_redflags)), [f"Address {i+1}" for i in range(len(df_redflags))])
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Cleanup

In [ ]:
# Close the driver connection
driver.close()
print("✓ Database connection closed")